<a href="https://colab.research.google.com/github/qhfks27/PyTorch/blob/master/RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
from torchsummary import summary

In [25]:
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 20
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)

다음 기기로 학습합니다: cuda


In [26]:
print("데이터 로딩중...")
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)
TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

데이터 로딩중...


In [27]:
trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)


vocab_size = len(TEXT.vocab)
n_classes = 2


print("[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어수]: %d [클래스] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

[학습셋]: 20000 [검증셋]: 5000 [테스트셋]: 25000 [단어수]: 46159 [클래스] 2


In [0]:
class LSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(LSTM, self).__init__()
        print("Building LSTM model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        c_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, (h_0, c_0))  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [0]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()


def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy


In [30]:
model = LSTM(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Building LSTM model...


In [31]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[이폭: 1] 검증 오차: 0.69 | 검증 정확도:49.80
[이폭: 2] 검증 오차: 0.69 | 검증 정확도:50.24
[이폭: 3] 검증 오차: 0.70 | 검증 정확도:49.64
[이폭: 4] 검증 오차: 0.70 | 검증 정확도:49.56
[이폭: 5] 검증 오차: 0.72 | 검증 정확도:51.02
[이폭: 6] 검증 오차: 0.72 | 검증 정확도:50.36
[이폭: 7] 검증 오차: 0.74 | 검증 정확도:53.18
[이폭: 8] 검증 오차: 0.72 | 검증 정확도:51.44
[이폭: 9] 검증 오차: 0.73 | 검증 정확도:51.48
[이폭: 10] 검증 오차: 0.75 | 검증 정확도:51.96
[이폭: 11] 검증 오차: 0.74 | 검증 정확도:53.18
[이폭: 12] 검증 오차: 0.73 | 검증 정확도:56.92
[이폭: 13] 검증 오차: 0.58 | 검증 정확도:73.62
[이폭: 14] 검증 오차: 0.71 | 검증 정확도:50.54
[이폭: 15] 검증 오차: 0.71 | 검증 정확도:52.98
[이폭: 16] 검증 오차: 0.70 | 검증 정확도:58.24
[이폭: 17] 검증 오차: 0.58 | 검증 정확도:73.62
[이폭: 18] 검증 오차: 0.50 | 검증 정확도:78.16
[이폭: 19] 검증 오차: 0.43 | 검증 정확도:83.28
[이폭: 20] 검증 오차: 0.45 | 검증 정확도:84.16


In [32]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.49 | 테스트 정확도: 80.98


In [33]:
print(model)

LSTM(
  (embed): Embedding(46159, 128)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(128, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)


In [0]:
def count_parameters(model):

    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [39]:
count = count_parameters(model)
print(count)

6304130
